In [12]:
import time, datetime, csv, dateutil
from itertools import islice
from os import remove
import os
import pandas as pd
import numpy as np
import warnings
#warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook as tqdm
import selenium as s
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

In [2]:
wd = webdriver.Firefox(executable_path=r'C:\geckodriver.exe')
URL = 'https://litnet.com/ru/reader/igra-nayavu-b128818'  
wd.get(URL)

In [18]:
def next_page(wd):
    time.sleep(1)
    
    try:
        banner = wd.find_element_by_xpath(".//div[@class='modal fade in']")
        action = webdriver.common.action_chains.ActionChains(wd)
        action.move_to_element_with_offset(banner, 150, 0) #move 150 pixels to the right to access Help link
        action.click()
        action.perform()
    except:
        pass
    
    time.sleep(2)
    next_page_class = "pull-right" #there are two buttons - for next page and next chapter. The latter has a suffix in class
    next_chapter_suffix = " chapter-btn"

    next_page_buttons = wd.find_elements_by_xpath(".//a[@class='%s']"%next_page_class)
    if len(next_page_buttons) == 2:
        next_page_buttons[0].click()
    else:
        next_chapter_buttons = wd.find_elements_by_xpath(".//a[@class='%s']"%(next_page_class+next_chapter_suffix))
        if len(next_chapter_buttons) == 2:
            next_chapter_buttons[0].click()
        elif len(next_chapter_buttons) == 0:
            print('The END')
            return -1
        else:
            print('there is something wrong...')
            return -1
    return 0

def read_page(wd, fname):
    cls = 'reader-text font-size-medium'
    
    template = wd.find_elements_by_xpath(".//div[@class='%s']"%cls)
    assert len(template)==1
    text = template[0].find_elements_by_xpath(".//p")
    
    page_list = []
    for par in text:
        page_list.append(par.text)
    
    page = '\n'.join(page_list) + '\n'
    
    f = open(fname,"a",encoding="utf-8") 
    f.write(page)
    f.close()
    
    return

def check_if_file_empty(fname):
    if os.stat(fname).st_size == 0:
        return
    else:
        f = open(fname,"w+",encoding="utf-8") 
        f.close()
    return

In [19]:
status = 0
fname = 'test.txt'
check_if_file_empty(fname)
while status == 0:
    read_page(wd,fname)
    status = next_page(wd)

The END
